In [ ]:
# Script to create date for Date Dimesion Landing

In [ ]:
# Import required libraries
import sys
from lib.spark_session import get_spark_session
from lib.utils import date_data, get_string_cols
from lib.job_control import insert_log
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp
from datetime import datetime

In [2]:
# Generate Spark Session
spark: SparkSession = get_spark_session("Date Landing")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

:: loading settings :: url = jar:file:/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e737da4a-ddd3-4602-96c4-e1509e8d81ee;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.1 in central
	found io.delta#delta-storage;2.1.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 253ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-core_2.12;2.1.1 from central in [default]
	io.delta#delta-storage;2.1.1 from central in [default]
	org.antlr#antlr4-runtime;4.

SPARK_APP: Spark UI - http://46346aae9d54:4040


In [3]:
# JOB Parameters
rundate = "20220101"

# Spark Configs
spark.conf.set("spark.sql.shuffle.partitions", 8)

23/02/03 11:04:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
# Define cols for date landing
_cols = ["date", "day", "month", "year", "day_of_month"]
_data = date_data(rundate, 2)

In [5]:
# Create Raw dataframe and get col info
df_raw = spark.createDataFrame(data=_data, schema=_cols)
print("SPARK_APP: Printing Landing Schema --")
df_raw.printSchema()

# Get Landing count
print("SPARK_APP: Landing data count - " + str(df_raw.count()))

SPARK_APP: Printing Landing Schema --
root
 |-- date: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- day_of_month: string (nullable = true)



SPARK_APP: Landing data count - 729


In [13]:
# Cast all columns to String
df_casted = df_raw.selectExpr(get_string_cols(spark, df_raw))
print("SPARK_APP: Casted all columns to String")

# Add audit columns
df_ld = df_casted.withColumn("insert_dt", current_timestamp())
print("SPARK_APP: Added AUDIT column")

# Get Final Layer count
print("SPARK_APP: Final layer data count - " + str(df_ld.count()))

SPARK_APP: Casted all columns to String
SPARK_APP: Added AUDIT column
SPARK_APP: Final layer data count - 729


In [15]:
# Write the date to landing layer
df_ld.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("edw_ld.dim_date_ld")

print("SPARK_APP: Data written to landing layer")

23/02/03 11:12:52 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `edw_ld`.`dim_date_ld` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


SPARK_APP: Data written to landing layer


In [17]:
# Add job details in JOB CONTROL
insert_log(spark, "edw_ld", "dim_date_ld", datetime.now())
print("SPARK_APP: Update JOB Control Log")

SPARK_APP: Update JOB Control Log


In [19]:
spark.sql("select * from edw.job_control").show(truncate=False)

+-----------+-----------+--------------------------+--------------------------+
|schema_name|table_name |max_timestamp             |insert_dt                 |
+-----------+-----------+--------------------------+--------------------------+
|edw_ld     |dim_date_ld|2023-02-03 11:16:36.679232|2023-02-03 11:16:39.436692|
+-----------+-----------+--------------------------+--------------------------+



In [ ]:
spark.stop()